In [2]:
# Install Dependencies 

! pip install beautifulsoup4
! pip install IProgress
! pip install ipywidgets
! pip install openpyxl
! pip install pandas
! pip install Pillow
! pip install requests
! pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [1]:
# Upload Spreadsheet which contains the product information that needs to be modified

import pandas as pd

def load_excel_to_dataframe(path):
    """
    Loads an Excel file at the given path into a Pandas DataFrame.

    Parameters:
    path (str): The file path to the Excel file.

    Returns:
    DataFrame: The loaded data as a Pandas DataFrame.
    """
    try:
        # Load the Excel file into a DataFrame
        df = pd.read_excel(path, engine='openpyxl')
        return df
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Use the function to load your .xlsx file
file_path = '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_data/elvato_products.xlsx'
df = load_excel_to_dataframe(file_path)

# Optional: Print the DataFrame to check if it loaded correctly
print(df.head)

<bound method NDFrame.head of                                         Handle  \
0                          introducing-sigfred   
1                          introducing-sigfred   
2                          introducing-sigfred   
3                          introducing-sigfred   
4     the-clancy-light-single-cylinder-pendant   
...                                        ...   
3904  study-bedroom-bedside-giraffe-table-lamp   
3905  study-bedroom-bedside-giraffe-table-lamp   
3906  study-bedroom-bedside-giraffe-table-lamp   
3907  study-bedroom-bedside-giraffe-table-lamp   
3908               spider-loft-wind-chandelier   

                                         Title  \
0                          Introducing Sigfred   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4     The Clancy Light Single Cylinder Pendant   
...                                        ...   
3904               

In [2]:
#Abatract Images from "Image Src" URL Column
## Fetch URL, rename to {fixture name} image {index} which is cataloguing all of our fixture images to a new directory file.

import os
import requests
from PIL import Image
from io import BytesIO
from tqdm.notebook import tqdm  # for a progress bar
import pandas as pd  # Ensure pandas is imported

def download_images(df, image_column, handle_column, target_folder):
    """
    Downloads all images from the specified column in the DataFrame and saves them to the target folder, prefixing the filenames with the handle.

    Parameters:
    df (DataFrame): The pandas DataFrame containing the image URLs and handles.
    image_column (str): The name of the column in the DataFrame containing the image URLs.
    handle_column (str): The name of the column in the DataFrame containing the handles.
    target_folder (str): The path to the folder where the images will be saved.
    """
    # Ensure the target folder exists
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Loop through the image URLs in the DataFrame
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Downloading images"):
        url = row[image_column]
        handle = row[handle_column]
        # Skip rows where the image URL or handle is missing or not a string
        if pd.isnull(url) or not isinstance(url, str) or pd.isnull(handle) or not isinstance(handle, str):
            # print(f"Skipping index {index}: No valid URL or handle provided.")
            continue

        # Attempt to download and save the image
        try:
            # Send a request to the image URL
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                # Open the image with PIL for format standardization and error checking
                image = Image.open(BytesIO(response.content))

                # Create a safe file name by replacing or removing invalid characters
                safe_handle = handle.replace("/", "_").replace("\\", "_")

                # Construct the filename using the handle as a prefix
                filename = os.path.join(target_folder, f"{safe_handle}_image_{index}.jpg")

                # Save the image data to the target file in a standard format
                image.save(filename, 'JPEG')
        except Exception as e:
            print(f"Failed to download {url}: {e}")

# Path to your DataFrame
file_path = '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_data/elvato_products.xlsx'
df = load_excel_to_dataframe(file_path)

# Specify the column name that contains the image URLs
image_column = 'Image Src'

# Specify the column name that contains the handle names
handle_column = 'Handle'  # Replace with the actual column name if different

# Specify the target folder for the downloaded images
target_folder = '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_images'

# Download the images
download_images(df, image_column, handle_column, target_folder)

In [3]:
# Abstract Product Titles to new CSV 

import pandas as pd
import os

def generate_product_title(handle):
    """
    Generate a product title from the handle.
    This function can be customized based on how you want to transform the handle into a title.

    Example transformation:
    - Replace certain separators with spaces
    - Capitalize each word
    - Add descriptive words or phrases based on certain keywords
    """
    # Replace common separators with spaces and capitalize each word
    title = handle.replace('-', ' ').replace('_', ' ').title()

    # Example of adding flair based on keywords (customize as needed)
    if 'vintage' in title.lower():
        title = "Classic " + title
    elif 'modern' in title.lower():
        title = title + " Design"
    elif 'kids' in title.lower():
        title = "Fun & Colorful " + title

    return title

def create_product_titles_csv(df, handle_column, output_dir, output_filename):
    """
    Creates a CSV file with unique product titles generated from the handles.

    Parameters:
    df (DataFrame): The pandas DataFrame containing the handles.
    handle_column (str): The name of the column in the DataFrame containing the handles.
    output_dir (str): The directory where the output CSV will be saved.
    output_filename (str): The name of the output CSV file.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Generate product titles
    df['Product Title'] = df[handle_column].apply(generate_product_title)

    # Remove duplicates and keep only the 'Product Title' column
    unique_titles_df = df[['Product Title']].drop_duplicates()

    # Save to CSV
    output_path = os.path.join(output_dir, output_filename)
    unique_titles_df.to_csv(output_path, index=False)
    print(f"File saved to {output_path}")

# Load your DataFrame (assuming it's already loaded as 'df')
# Call the function with appropriate parameters
create_product_titles_csv(df, 'Handle', '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_revisions', 'handle_product_titles.csv')

File saved to /Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_revisions/handle_product_titles.csv


In [4]:
# Abstract Product Descriptions to new CSV 

import pandas as pd
import os
from bs4 import BeautifulSoup

def clean_html_description(html_content):
    """
    Cleans and processes HTML content to extract a cleaner, plain text description.
    Uses Beautiful Soup for robust HTML parsing.

    Parameters:
    html_content (str): The HTML content from the 'Body (HTML)' column.

    Returns:
    str: A cleaned and processed version of the HTML content.
    """
    if pd.isnull(html_content):
        return ""  # Return an empty string for NaN values

    # Parse and clean the HTML content using Beautiful Soup
    soup = BeautifulSoup(html_content, "html.parser")
    text = soup.get_text(separator=' ')

    # Replace common HTML entities
    text = text.replace('&amp;', '&').replace('&nbsp;', ' ')
    return text.strip()

def create_product_descriptions_csv(df, body_html_column, output_dir, output_filename):
    """
    Creates a CSV file with product descriptions extracted and cleaned from the 'Body (HTML)' column.

    Parameters:
    df (DataFrame): The pandas DataFrame containing the HTML descriptions.
    body_html_column (str): The name of the column in the DataFrame containing the HTML content.
    output_dir (str): The directory where the output CSV will be saved.
    output_filename (str): The name of the output CSV file.
    """
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Clean and extract descriptions
    df['Product Description'] = df[body_html_column].apply(clean_html_description)

    # Remove duplicates and keep only the 'Product Description' column
    unique_descriptions_df = df[['Product Description']].drop_duplicates()

    # Save to CSV
    output_path = os.path.join(output_dir, output_filename)
    unique_descriptions_df.to_csv(output_path, index=False)
    print(f"File saved to {output_path}")

# Assuming your DataFrame is already loaded as 'df'
# Call the function with appropriate parameters
create_product_descriptions_csv(df, 'Body (HTML)', '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_revisions', 'Body-(HTML)_product_descriptions.csv')

File saved to /Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_revisions/Body-(HTML)_product_descriptions.csv


In [5]:
# Abstract Product 'Handle-tags' to new CSV 

import pandas as pd
import os

def extract_and_save_handles(input_path, output_dir, output_filename):
    """
    Extracts the 'Handle' column from the specified Excel file and saves it to a new CSV file.

    Parameters:
    input_path (str): The path to the Excel file containing the product data.
    output_dir (str): The directory where the output CSV will be saved.
    output_filename (str): The name of the output CSV file.
    """
    # Load the data from the Excel file
    df = pd.read_excel(input_path, engine='openpyxl')

    # Extract the 'Handle' column
    handles_df = df[['Handle']].drop_duplicates()

    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the full output path
    output_path = os.path.join(output_dir, output_filename)

    # Save the handles to a new CSV file
    handles_df.to_csv(output_path, index=False)

    print(f"File saved to {output_path}")

# Set the parameters for the function
input_path = '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_data/elvato_products.xlsx'
output_dir = '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_revisions'
output_filename = 'handle_tags.csv'

# Call the function
extract_and_save_handles(input_path, output_dir, output_filename)

File saved to /Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_revisions/handle_tags.csv


In [6]:
import pandas as pd
import os

def extract_and_save_skus(input_path, output_dir, output_filename):
    """
    Extracts the 'Variant SKU' column from the specified Excel file and saves it to a new CSV file.

    Parameters:
    input_path (str): The path to the Excel file containing the product data.
    output_dir (str): The directory where the output CSV will be saved.
    output_filename (str): The name of the output CSV file.
    """
    # Load the data from the Excel file
    df = pd.read_excel(input_path, engine='openpyxl')

    # Extract the 'Variant SKU' column
    skus_df = df[['Variant SKU']].drop_duplicates()

    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the full output path
    output_path = os.path.join(output_dir, output_filename)

    # Save the 'Variant SKU' data to a new CSV file
    skus_df.to_csv(output_path, index=False)

    print(f"File saved to {output_path}")

# Set the parameters for the function
input_path = '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_data/elvato_products.xlsx'
output_dir = '/Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_revisions'
output_filename = 'cj-sku_numbers.csv'

# Call the function
extract_and_save_skus(input_path, output_dir, output_filename)

File saved to /Users/matthewsimon/Documents/GitHub/acdc.medusa/Data/source_revisions/cj-sku_numbers.csv
